# Chapter 10: Robustness

## Item 80: Take Advantage of Each Block in `try/except/else/finally`

In [ ]:
def erics_cool_file_reader(filename):
    print("Step 1: lets gooo")
    handle = open(filename)
    try:
        print("Step 2: read the file")
        return handle.read()
    finally:
        print("Step 3: close the file")
        handle.close()


# This is essentially what `with open` is doing
erics_cool_file_reader("packages.csv")

In [ ]:
# THE ORDER OF EXECUTION

throwaway_txt = "throwaway.txt"
with open(throwaway_txt, "wb") as f:
    f.write(b"\xf1\xf2\xf3\xf4")

data = erics_cool_file_reader("foop")

Note the order of operations. An exception did occur, but "finally" happened before the exception was propagated.

So-called finally! Its a conspiracy!!!

Bonus quiz: Why did the author not put `open` in the `try/finally` block?

In [ ]:
# Else blocks
# These run when an exception didn't happen
import json


def get_some_json(data, key):
    try:
        print("READING")
        res = json.loads(data)
    except ValueError:
        print("WHOOPS")
        raise  # Author raises a KeyError, possibly for the caller's benefit?
    else:
        print("Looking!")
        return res[key]


assert get_some_json('{"foo": "bar"}', "foo") == "bar"

In [ ]:
# We're only catching errors with loading json, propagate out the rest
get_some_json('{"foo": "bar"}', "baz")

In [ ]:
# Some bad json
get_some_json('{"foo" "bar"}', "foo")

In [ ]:
# PUT THEM ALL TOGETHER

UNDEFINED = object()


def divider(path):
    print("Open er up")
    handle = open(path, "r+")

    try:
        print("Read!!!")
        data = handle.read()

        print("Parse!!!")
        data = json.loads(data)

        print("Divide!!!")
        value = data["numerator"] / data["denominator"]
    except ZeroDivisionError:
        print("File not found")
        return UNDEFINED
    else:
        print("Calculate!!!")
        data["result"] = value
        result = json.dumps(data)
        handle.seek(0)
        handle.write(result)
        return value
    finally:
        print("Close!!!")
        handle.close()

In [ ]:
# happy path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1, "denominator": 10}')

assert divider(temp_path) == 0.1

with open(temp_path, "r+") as f:
    print(f.read())

In [ ]:
# zero division path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1, "denominator": 0}')

assert divider(temp_path) is UNDEFINED

with open(temp_path, "r+") as f:
    print(f.read())

In [ ]:
# Bad data path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1 lol what}')

assert divider(temp_path) == 0.1

with open(temp_path, "r+") as f:
    print(f.read())

## Item 81: `assert` Internal Assumptions and `raise` Missed Expectations
Note: several linters will warn you about using `assert`

In [ ]:
list_a = [1, 2, 3]
assert list_a, "MUST NOT BE EMPTY!!!"

list_b = []
assert list_b, "MUST NOT BE EMPTY!!!"

In [ ]:
# there's also raise
class EmptyError(Exception):
    pass


list_c = []
if not list_c:
    raise EmptyError("list_c is empty")

In [ ]:
try:
    raise EmptyError("raised it")
except EmptyError as e:
    print(f"caught {e}")

Note that `assert` raises `AssertionError`

So, when to use what?

### `raises`
This is considered part of a function's interface. The caller is expected to do something with it

### `assert`
This is not supposed to be part of the interface. It's used to verify assumptions in an implementation. It helps new readers of code understand that code better. Self-documenting. etc. Errors raised via assert can be treated as system problems, instead of "the caller made a mistake". "Our assumption was wrong"

In [ ]:
class MovieRating:
    def __init__(self, max_rating):
        assert 0 <= max_rating <= 10, (
            "Rating must be between 0 and 10"
        )  # assumption: the caller validated the max rating
        self.max_rating = max_rating

    def rate(self, rating):
        if not 0 <= rating <= self.max_rating:
            raise ValueError(
                f"Rating must be between 0 and {self.max_rating}"
            )  # error: the caller gave too high a rating

In [ ]:
rater = MovieRating(11)  # our code is bugged

In [ ]:
rater = MovieRating(10)
rater.rate(400)  # their code is bugged

Still fluffy?

That's okay, this is one of those "at your own discretion" areas.

My only warning around this (and the author warns this too) is to not catch `AssertionError`! Don't do it! It defeats the point. This includes not doing a high-level `except Exception`, because `AssertionError` _is_ an `Exception`.

### Things to Remember:
- The raise statement can be used to report expected error conditions back to the caller
- Exceptions raised by a function/method are part of its explicit interface
- The assert statement should be used to verify a programmers assumptions about code
- Dont catch/handle assertion errors! They indicate a bug!!!

### Side note: custom exceptions
stuff hes talking about here hints at something I _hope_ he talks about later (item 121): when building your own API, its usually better to make your own exception hierarchy, instead of using the builtins. That way you can have a top-level `except` for your module's code, without accidentally catching stuff you shouldn't (assertion errors!!!)

## Item 82: Consider `contextlib` and `with` Statements for Reusable `try/finally` Behavior

In [ ]:
from threading import Lock
lock = Lock()

with lock:
    print("I'm running with a lock! Pretend theres threading code")

In [ ]:
# Basically what's happening:

lock.acquire()
try:
    print("I'm running with a lock! Pretend theres threading code")
finally:
    lock.release()

Can save oneself repetitive code by using `with` instead of the frequent `try/finally`

In [ ]:
# lets use it

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def my_func():
    logger.debug("some msg")
    logger.info("Info about stuff!")
    logger.debug("more msg")

my_func()

In [ ]:
# Oh no no debug info! Maybe we set our app to that level. But what if we want a PARTICULAR area to always log its debug messages, but we don't care about other debug?

def my_func_2():
    current_level = logger.getEffectiveLevel()
    logger.setLevel(logging.DEBUG)
    logger.debug("some msg")
    logger.info("Info about stuff!")
    logger.debug("more msg")
    logger.setLevel(current_level)

my_func_2()

In [ ]:
# Gross if we have to scale it up right?

from contextlib import contextmanager

@contextmanager
def debug_logging_on():
    current_level = logger.getEffectiveLevel()
    logger.setLevel(logging.DEBUG)
    try:
        yield
    finally:
        logger.setLevel(current_level)

In [ ]:
# Behold
with debug_logging_on():
    my_func()

my_func()

We can get variables from the context with `as`.

```python
handle = open("my_file.txt", "w")
try:
    handle.write("cowabunga")
finally:
    handle.close()

with open("my_file.txt", "w") as handle:
    handle.write("cowabunga")
```

In [ ]:
# do it ourselves

@contextmanager
def log_with(level, name):
    my_logger = logging.getLogger(name)
    current_level = my_logger.getEffectiveLevel()
    my_logger.setLevel(level)
    try:
        yield my_logger  # this is what is sent out on the `as` part
    finally:
        logger.setLevel(current_level)

with log_with(logging.DEBUG, "my_module") as loggeroni:
    loggeroni.debug("some msg")
    loggeroni.info("Info about stuff!")
    loggeroni.debug("more msg")
    logger.debug("The other logger! We shouldn't see this message")

In [ ]:
# just for fun, lets get confusing
with log_with(logging.DEBUG, "my_module") as loggeroni:
    with debug_logging_on():
        loggeroni.debug("some msg")
        loggeroni.info("Info about stuff!")
        loggeroni.debug("more msg")
        logger.debug("The other logger! We shouldn't see this message")

### Things to Remember
- `with` allows you some nice reusable try/finally behavior
- `contextlib`'s `contextmanager` makes it easy to build with-capable functions
- The value one yields from a context manager is what is provided to the `as` statement.

## Item 83: Always Make `try` Blocks as Short as Possible
It's so easy to make these huge.

Try to keep try blocks close to their error unit. Put only one source of **expected errors** in each try block

In [ ]:
def shitty_func():
    raise ValueError("I'm a bad function")

def wacky_func():
    raise ValueError("el oh el i'm so crazy ;P")

try:
    shitty_func()
    wacky_func()
except ValueError as e:
    print("somebody goofed")

But who goofed?!

Obviously we can reason about this example, but lets say these were real functions and it wasn't obvious if they'd throw or not.

In that case, we have to think more about "Who goofed"

In [ ]:
try:
    shitty_func()
except ValueError as e:
    print("Shitty_Func definitely goofed!")
else:
    wacky_func() # will also goof, but thats not the point

If you can, try to have as little in the try block as possible. Move other things to `else:` or their own try block

## Item 84: Beware of Exception Variables Disappearing

In a lot of places in python there are `blah as foo` statements, including exceptions: `except FooError as foo`. But unlike many others, the exception var does not live on past the exception block.

In [ ]:
try:
    shitty_func()
except ValueError as e:
    print("It wont stop goofin!!!")

print(f"whats good {e=}")

In [ ]:
# Oh that didn't work, well what about finally?

try:
    shitty_func()
except ValueError as e:
    print("It wont stop goofin!!!")
finally:
    print(f"whats good {e=}")

In [ ]:
# if we DO need the variable outside the except block, its straight-forward:

goof_meter = "unknown goofs"

try:
    wacky_func()
except ValueError as e:
    goof_meter = "some goofs"
except AttributeError as e:
    goof_meter = "some goofs"
else:
    goof_meter = "no goofs"
finally:
    print(f"Approximate goof rate: {goof_meter}")

## Item 85: Beware of Catching the `Exception` Class
Oh boy, its the big one. Every linter in every language (that has exceptions) warns about this.

In [ ]:
# Let's do some goof analysis

def load_data(raw_goofs):
    with open(raw_goofs) as f:
        return f.read()

def analyze_goofs(goofs):
    return goofs.count("goof")

def run_goof_report(path):
    data = load_data(path)
    goofs = analyze_goofs(data)
    return goofs

In [ ]:
run_goof_report("daily_goofs-08-16-2025.txt")

In [ ]:
# Aw dang it!
try:
    run_goof_report("daily_goofs-08-16-2025.txt")
except FileNotFoundError as e:
    print(f"Goof file not found: {e}")

In [ ]:
# solved! Oh, but now we also have other stuff to add
def run_goof_report(path):
    shitty_func()  # New behavior with its own error chain
    data = load_data(path)
    goofs = analyze_goofs(data)
    return goofs

try:
    run_goof_report("daily_goofs-08-16-2025.txt")
except Exception as e:  # Solved once and for all!!!
    print(f"Somebody really goofed: {e}")

In [ ]:
def run_goof_report(path):
    bug_in_ur_code()  # more new functionality introduced, shittier than before!
    shitty_func()
    data = load_data(path)
    goofs = analyze_goofs(data)
    return goofs

try:
    run_goof_report("daily_goofs-08-16-2025.txt")
except Exception as e:
    print(f"Somebody really goofed: {e}")

The problem here, is nobody was goofin, this was a goofin bug in our goof code! This sort of thing should bubble up to the top for us to see! Not be handled and ignored, as it represents bad, unrecoverable state!

In [ ]:
# Let's mitigate
try:
    run_goof_report("daily_goofs-08-16-2025.txt")
except Exception as e:
    print("Somebody really goofed:", type(e), e)

## Item 86: Understand the Difference Between Exception and BaseException

In [ ]:
import inspect

print(inspect.getmro(BaseException))
print(inspect.getmro(Exception))

In [ ]:
try:
    raise Exception("GOOFED")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    raise KeyboardInterrupt("KEYBOARD GOOF")
except Exception as e:
    print(f"Exception: {e}")

WHAT?! HOW?!

In [ ]:
inspect.getmro(KeyboardInterrupt)

Yes.

Certain exceptions inherit from BaseException instead of Exception. SystemExit, KeyboardInterrupt, etc.

Basically, if its used by the runtime itself, it will inherit from BaseException instead of Exception. And because of this, one should avoid catching BaseException

In [ ]:
import random

try:
    if random.random() < 0.5:
        raise SystemExit("I'm not an error")
    else:
        raise Exception("Legit goof!")
except BaseException as be:
    # a greater sin than catching Exception!
    print(f"BaseException: {be}")

It's not necessarily wrong to catch BaseException, just dangerous.

### Things to Remember

- For internal behaviors, Python sometimes raises BaseException child classes, which will skip except clauses that only handle the Exception base class.
- try/finally statements, with statements, and similar constructs properly handle raised BaseException child classes without extra effort.
- There are legitimate reasons to catch BaseException and related classes, but doing so can be error prone.

## Item 87: Use `traceback` for Enhanced Exception Reporting

In [ ]:
def goofable(message):
    assert False, message

def ungoofable_trust_me_bro(message):
    goofable(message)

ungoofable_trust_me_bro("This will never goof!")

Works great, but think about concurrent/parallelism: do we want an error in one to crash the whole program? Possibly no.

Let's solve it with a broad exception handler!!!

In [ ]:
def handle(msg):
    try:
        goofable(msg)
    except BaseException as e:
        print(repr(e))

handle("Don't goof damn it!")

Remember, this follows some of the advice from before: if you _have_ to have a blanket handler like this, at least show the exception type.

But this has a problem: where the hell is the traceback?!

In [ ]:
import traceback

def handle2(msg):
    try:
        goofable(msg)
    except BaseException as e:
        traceback.print_tb(e.__traceback__)
        print(repr(e))

handle2("Don't goof damn it!")

In [ ]:
def handle3(msg):
    try:
        goofable(msg)
    except BaseException as e:
        stack = traceback.extract_tb(e.__traceback__)
        for frame in stack:
            print(frame.name)
        print(repr(e))

handle3("STOP GOOFING I BEG OF YOU!!")

### Things to Remember

- When an unhandled exception propagates up to the entry point of a Python program, the interpreter will print a nicely formatted list of the stack frames that caused the error.
- In highly concurrent programs, exception tracebacks are often not printed in the same way, making errors more difficult to understand and debug.
- The traceback built-in module allows you to interact with the stack frames from an exception and process them in whatever way you see fit (i.e., to aid in debugging).

## Item 88: Consider Explicitly Chaining Exceptions to Clarify Tracebacks

In [ ]:
my_dict = {}
my_dict["does_not_exist"]

In [ ]:
# Lets make some handling!
class MissingError(Exception):
    ...
def lookup(a_dict, key):
    try:
        return a_dict[key]
    except KeyError:
        raise MissingError(
            f"Key '{key}' not found in dictionary"
        )

lookup(my_dict, "does_not_exist")

In [ ]:
# Exceptions have a "context", BEHOLD

try:
    lookup(my_dict, "does_not_exist")
except MissingError as e:
    print("Second:", repr(e))
    print("First: ", repr(e.__context__))

In [ ]:
my_dict["my key 1"] = 123
lookup(my_dict, "my key 1")

In [ ]:
class ServerMissingKeyError(Exception):
    ...

the_server = {"foo": "bar"}

def fetch_from_server(key):
    print("fetching from server")
    try:
        return the_server[key]
    except KeyError:
        raise ServerMissingKeyError(
            f"Key '{key}' not found on server"
        )

def lookup2(a_dict, key):
    try:
        return a_dict[key]
    except KeyError:
        a_dict[key] = fetch_from_server(key)
        raise MissingError(
            f"Key '{key}' not found in dictionary"
        )

In [ ]:
some_dict = {"bar": "baz"}
lookup2(some_dict, "foo")

In [ ]:
lookup2(some_dict, "foo")

In [ ]:
# But what about when the server doesn't have it either?
lookup2(some_dict, "baz")

Well now we get a ServerMissingKeyError instead of a MissingError! THE CONTRACT IS BROKEN!

INSTEAD LETS CHAIN

In [ ]:
def lookup3(a_dict, key):
    try:
        return a_dict[key]
    except KeyError as e:
        try:
            result = fetch_from_server(key)
        except ServerMissingKeyError:
            raise MissingError(
                f"Key '{key}' not found in dictionary"
            ) from e
        else:
            a_dict[key] = result
            return a_dict[key]

In [ ]:
lookup3(some_dict, "baq")

### Things to Remember

- When an exception is raised from inside an except clause, the original exception for that handler will always be saved to the newly raised Exception value’s __context__ attribute.
- The from clause in the raise statement lets you explicitly indicate—by setting the __cause__ attribute—that a previously raised exception is the cause of a newly raised one.
- Explicitly chaining one exception from another will cause Python to only print the supplied cause (or lack thereof) instead of the automatically chained exception.

## Item 89: Always Pass Resources into Generators and Have Callers Clean Them Up Outside

In [ ]:
# finally doesn't preempt exits

def my_func():
    try:
        return 123
    finally:
        print("Finally my_func")

print("Before")
print(my_func())
print("After")

In [ ]:
# But what about generators?

def my_generator():
    try:
        yield 10
        yield 20
        yield 30
    finally:
        print("Finally my_generator")

print("Before")
for i in my_generator():
    print(i)

print("After")

Pause, so what's the difference? Shouldn't `finally` go before the exit? Yes! But unlike `return`, `yield` is not an exit! Generators exit condition is the `StopIteration` exception.

And that is an important detail! Unlike `return`, `StopIteration` isn't guaranteed! A generator can like.. you know... just keep going.

In [ ]:
# Behold, we don't get a finally!

gooferator = my_generator()
print("Before")
print(next(gooferator))
print(next(gooferator))
print("After")

The generator didn't actually reach an exit condition, and therefore, `finally` didn't trigger.

The GC might come around and nab it, if all references are gone:

In [ ]:
import gc
del gooferator
gc.collect()

:amaze:

When python's GC cleans up the generator, it sends `GeneratorExit` exception. This exception causes the generator to return and clear its stack.

In [ ]:
# PROOF

def catching_generator():
    try:
        yield 40
        yield 50
        yield 60
    except BaseException as e:  # Catches GeneratorExit
        print("Catching handler", type(e), e)
        raise

gooferator = catching_generator()

print("Before")
print(next(gooferator))
print(next(gooferator))
print("After")

del gooferator
gc.collect()

`GeneratorExit` isn't _actually_ handled by our generator, its handled by the garbage collector.

Can we break it?

In [ ]:
def broken_generator():
    try:
        yield 70
        yield 80
    except BaseException as e:
        print("Broken handler", type(e), e)
        raise RuntimeError("This should be a disaster and stop everything")
    finally:
        print("Finally handler")

gooferator = broken_generator()
print("Before")
print(next(gooferator))
print("After")
del gooferator
gc.collect()
print("Still going")


Note, that the `raise`, which _should_ have halted the program, did not. Again, the `gc` handles the `GeneratorExit`. What happened here, is we interrupted its own routine by injecting that `RuntimeError`. The `gc` can't just _crash_, so instead it swallows the exception, and continues on its merry way.

One CAN still do this in a defensive and robust way... but why, when there's an easy alternative?

In [ ]:
# Don't let generators manage file handles and the like

def sing_to_me(path):
    """Yield lines from a file

    This is so super safe by using that with statement!!! DONT QUESTION IT!!!
    """
    try:
        with open(path) as handle:
            for line in handle:
                yield line
    finally:
        print("Done singing!")

In [ ]:
singerator = sing_to_me("sayitaintso.txt")

for line in singerator:
    print(line)
    if "Wrestle with Jimmy" in line:
        break

In [ ]:
# but wait, where was the finally?????

del singerator
gc.collect()

Damn it! We didn't solve this goofy delayed behavior! I WANT THE RESOURCES CLEANED UP NOW!!!

Seriously though, imagine this was something more high-stakes, like a mutex lock.

![](deadlock.png)

In [ ]:
# Okay, instead lets not let the generator manage the handle AT ALL

def sing_to_me_2(handle):
    try:
        for line in handle:
            yield line
    finally:
        print("Done singing!")

In [ ]:
with open("sayitaintso.txt") as handle:
    singerator = sing_to_me_2(handle)
    for line in singerator:
        print(line)
        if "Wrestle with Jimmy" in line:
            break

print("Is handle closed?", handle.closed)

### Things to Remember

- In normal functions, finally clauses are executed before values are returned, but in generator functions, finally clauses are only run after exhaustion, when the StopIteration exception is raised.
- In order to prevent memory leaks, the garbage collector injects GeneratorExit exceptions into unreferenced, partially iterated generators to cause them to exit and release resources.
- Due to this behavior, it’s often better to pass resources (like files and mutexes) into generator functions instead of relying on them to allocate and clean up the resources properly.

## Item 90: Never Set `__debug__` to `False`

Remember when we talked about `assert`?

In [ ]:
n = 3
assert n % 2 == 0, f"{n=} not even"

In [ ]:
# This is functionally equivalent to
if __debug__:
    if not (n % 2 == 0):
        raise AssertionError(f"{n=} not even")

In [ ]:
# this wont work
__debug__ = False

Somebody remind Eric to hover his mouse over `__debug__` in PyCharm.

Yep! `__debug__` can only be changed by the `-O` interpreter flag. It will stay the same for the duration of the program.

`-O` is the "optimized mode" flag. And this is _basically_ all it does. And in the runtime itself, all this does is disable `asserts`.

TL;DR: This is a lame way to get a performance boost. There's better ways (which are coming in the next chapter!)

## Things to Remember

- By default, the `__debug__` global built-in variable is True, and Python programs will execute all `assert` statements.
- The `-O` command-line flag can be used to set `__debug__` to False, which causes assert statements to be ignored.
- Having `assert` statements present can help narrow the cause of bugs even when the assertions themselves haven’t failed.

## Item 91: Avoid `exec` and `eval` Unless You’re Building a Developer Tool

In [ ]:
eval("1 + 2")

Yes, you saw that right.

`eval` and `exec` are the ultimate expression of Python's "screw it, do whatever you want in the runtime!" attitude. They let you execute arbitrary code from strings.

_They let you execute arbitrary code from strings._

In [ ]:
# Eval is for single expressions
eval("print('foobar!')")

In [ ]:
# But won't work with multiple lines

eval(
    """
if True:
    print('okay')
else:
    print('no')
"""
)

In [ ]:
# This is where exec comes in
exec(
    """
if True:
    print('okay')
else:
    print('no')
"""
)

You can see why this is powerful. Imagine a python program that can write and execute a python program.

Imagine a code agent willy-nilly using these.

### WARNING: Boring Eric Anecdote Time!
#### The Time Eric Used Exec in Real Code

In [ ]:
# exec always returns None. To get data in and out of it, we have to use global and local scope variables.
global_scope = {"my_condition": False}
local_scope = {}
exec(     """
if my_condition: # see the global scope?
    x = 'yes'
else:
    x = 'no'
""",
    global_scope,
    local_scope,
)
print(local_scope)

Typically, finding `eval` and `exec` in a codebase is a red flag. And some linters will let you know it!

Examples of _proper_ usage? REPLs, Notebooks (like jupyter!), code generation, and so on.

## Things to Remember

- eval allows you to execute a string containing a Python expression and capture its return value.
- exec allows you to execute a block of Python code and affect variable scope and the surrounding environment.
- Due to potential security risks, these features should be used rarely or never, limited only to improving the development experience.